##### Imports and Settings

In [1]:
import pandas as pd
import numpy as np
import requests
from functools import reduce
import matplotlib.pyplot as plt
import pickle
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
import sys
sys.path.append("../../../Functions and Dictionaries") # Adds higher directory to python modules path
import geodict
GNRC = geodict.GNRC
KY = geodict.KY
censusplaces = geodict.censusplaces
tofullcensus = geodict.tofullcensus
geotogeoid = geodict.geotogeoid
shorttnplaces = geodict.shorttnplaces
shortkyplaces = geodict.shortkyplaces
GNRC = geodict.GNRC
KY = geodict.KY
censusplaces = geodict.censusplaces
import tbrstuff
communitycollegescounties = tbrstuff.communitycollegescounties
tcatcampuscounties = tbrstuff.tcatcampuscounties
import sqlite3 as sq
#functions
def percent(x, y):
        try:
            return ((x/y)*100)
        except ZeroDivisionError:
            return 0
def percentchange(x, y):
    try:
        return ((x - y)*100/y)
    except ZeroDivisionError:
        return 0
def realchange(x, y):
    return x-y
#calculate real and percent change between all columns for all possible time frames
def calculate_changes(df, columns, time_frames, years):
    for column in columns:
        for time_frame in time_frames:
            start_year, end_year = time_frame.split('-')
            df[f'{column} % Change', 'None', f'{time_frame}'] = percentchange(df[(column, int(end_year), 'None')], df[(column, int(start_year), 'None')])
            df[f'{column} Change', 'None', f'{time_frame}'] = (df[(column, int(end_year), 'None')] - df[(column, int(start_year), 'None')])

    return df
#generate all possible time frames from a list of years
def generate_time_frames(years):
    time_frames = []
    for i in range(len(years)-1):
        for j in range(i+1, len(years)):
            time_frames.append(f"{years[i]}-{years[j]}")
    return time_frames

In [2]:
cip = pd.read_csv('../../../Heirarchies Folder/data/CIPCrosswalk.csv', dtype = str)

In [4]:
cip.head(2)

,Program Family,Two Digit,Program Category,Four Digit,Program,Six Digit
0,Agricultural/Animal/Plant/Veterinary Science a...,01,"Agriculture, General",01.00,"Agriculture, General",01.0000
1,Agricultural/Animal/Plant/Veterinary Science a...,01,Agricultural Business and Management,01.01,"Agricultural Business and Management, General",01.0101


In [60]:
df = pd.read_csv('../../Data Downloads/TBR_TCATS_201920_202122.csv', dtype = str)

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1431 entries, 0 to 1430
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Year              1431 non-null   object
 1   Program           1431 non-null   object
 2   Six Digit         1431 non-null   object
 3   Program Category  1431 non-null   object
 4   Four Digit        1431 non-null   object
 5   Program Family    1431 non-null   object
 6   Two Digit         1431 non-null   object
 7   Institution Type  1431 non-null   object
 8   Institution       1431 non-null   object
 9   Campus            1431 non-null   object
 10  County            1431 non-null   object
 11  FIPS              1431 non-null   object
 12  Certificate       1431 non-null   object
 13  Diploma           1431 non-null   object
 14  Total Awards      1431 non-null   object
dtypes: object(15)
memory usage: 167.8+ KB


In [62]:
df.head()

,Year,Program,Six Digit,Program Category,Four Digit,Program Family,Two Digit,Institution Type,Institution,Campus,County,FIPS,Certificate,Diploma,Total Awards
0,2019-20,Computer Systems Networking and Telecommunicat...,11.0901,Computer Systems Networking and Telecommunicat...,11.09,Computer and Information Sciences and Support ...,11,TCAT,Athens,Main Campus,"McMinn County, Tennessee",47107,0,10,10
1,2019-20,Industrial Mechanics and Maintenance Technolog...,47.0303,Heavy/Industrial Equipment Maintenance Technol...,47.03,Mechanic and Repair Technologies/Technicians,47,TCAT,Athens,Main Campus,"McMinn County, Tennessee",47107,8,8,16
2,2019-20,Autobody/Collision and Repair Technology/Techn...,47.0603,Vehicle Maintenance and Repair Technologies/Te...,47.06,Mechanic and Repair Technologies/Technicians,47,TCAT,Athens,Main Campus,"McMinn County, Tennessee",47107,1,10,11
3,2019-20,Automobile/Automotive Mechanics Technology/Tec...,47.0604,Vehicle Maintenance and Repair Technologies/Te...,47.06,Mechanic and Repair Technologies/Technicians,47,TCAT,Athens,Main Campus,"McMinn County, Tennessee",47107,7,4,11
4,2019-20,Machine Tool Technology/Machinist,48.0501,Precision Metal Working,48.05,Precision Production,48,TCAT,Athens,Main Campus,"McMinn County, Tennessee",47107,0,11,11


In [90]:
recent = pd.read_csv('../../Data Downloads/TBR_TCAT_20222023.csv', dtype = str)

In [91]:
recent.head()

,award_count,TCAT_School,Award_Type,ReportYr,location_name,cip
0,12,Athens,Certificate,2022-23,Main Campus,480508
1,1,Athens,Certificate,2022-23,Main Campus,480501
2,1,Athens,Certificate,2022-23,Main Campus,470604
3,4,Athens,Diploma,2022-23,Main Campus,520402
4,5,Athens,Diploma,2022-23,Main Campus,470603


In [92]:
recent_pivot = pd.pivot(recent, index = ['TCAT_School', 'ReportYr', 'cip', 'location_name'], columns = 'Award_Type', values = 'award_count')

In [93]:
recent = recent_pivot.reset_index(drop = False)
recent = recent.rename_axis(None, axis=1)

In [95]:
cols = ['Certificate', 'Diploma']
recent = recent.fillna(0)
recent[cols] = recent[cols].astype(int)
recent['Total Awards'] = recent['Certificate'] + recent['Diploma']

In [96]:
recent.head()

,TCAT_School,ReportYr,cip,location_name,Certificate,Diploma,Total Awards
0,Athens,2022-23,110901,Main Campus,0,2,2
1,Athens,2022-23,470303,Main Campus,0,13,13
2,Athens,2022-23,470303,McMinn Higher Education Center Extension Campus,0,10,10
3,Athens,2022-23,470603,Main Campus,0,5,5
4,Athens,2022-23,470604,Main Campus,1,7,8


In [97]:
#cip = cip.drop(columns = ['Two Digit', 'Four Digit'])

In [98]:
recent['Two Digit'] = recent['cip'].str.slice(start = 0, stop = 2)
recent['Four Digit'] = recent['Two Digit'] + '.' + recent['cip'].str.slice(start = 2, stop = 4)
recent['Six Digit'] = recent['Four Digit'] + recent['cip'].str.slice(start = 4)

In [99]:
recent = recent.merge(cip, on = 'Six Digit')

In [100]:
recent = recent.rename(columns = {'TCAT_School': 'Institution',
                                  'ReportYr': 'Year', 
                                  'location_name': 'Campus'
                                 })
recent = recent.drop(columns = 'cip')
recent['Institution Type'] = 'TCAT'
recent['Inst_Campus'] = recent['Institution'] + ', ' + recent['Campus']

In [101]:
recent['County'] = recent['Inst_Campus'].map(tcatcampuscounties)
recent['FIPS'] = recent['County'].map(geotogeoid)

In [102]:
boop = recent['FIPS'].str.split(pat = 'US', expand = True)
recent['FIPS'] = boop[1].str.strip()

In [103]:
recent = recent[['Year', 'Program', 'Six Digit', 'Program Category', 'Four Digit', 'Program Family', 'Two Digit', 'Institution Type', 'Institution', 
         'Campus', 'County', 'FIPS', 'Certificate', 'Diploma', 'Total Awards'
        ]]

In [104]:
recent.head()

,Year,Program,Six Digit,Program Category,Four Digit,Program Family,Two Digit,Institution Type,Institution,Campus,County,FIPS,Certificate,Diploma,Total Awards
0,2022-23,Computer Systems Networking and Telecommunicat...,11.0901,Computer Systems Networking and Telecommunicat...,11.09,Computer and Information Sciences and Support ...,11,TCAT,Athens,Main Campus,"McMinn County, Tennessee",47107,0,2,2
1,2022-23,Computer Systems Networking and Telecommunicat...,11.0901,Computer Systems Networking and Telecommunicat...,11.09,Computer and Information Sciences and Support ...,11,TCAT,McKenzie,Main Campus,"Carroll County, Tennessee",47017,6,5,11
2,2022-23,Computer Systems Networking and Telecommunicat...,11.0901,Computer Systems Networking and Telecommunicat...,11.09,Computer and Information Sciences and Support ...,11,TCAT,Paris,Main Campus,"Henry County, Tennessee",47079,4,11,15
3,2022-23,Computer Systems Networking and Telecommunicat...,11.0901,Computer Systems Networking and Telecommunicat...,11.09,Computer and Information Sciences and Support ...,11,TCAT,Pulaski,Main Campus,"Giles County, Tennessee",47055,5,6,11
4,2022-23,Industrial Mechanics and Maintenance Technolog...,47.0303,Heavy/Industrial Equipment Maintenance Technol...,47.03,Mechanic and Repair Technologies/Technicians,47,TCAT,Athens,Main Campus,"McMinn County, Tennessee",47107,0,13,13


In [105]:
dfs = [df, recent]
tcats = pd.concat(dfs)

In [107]:
tcats.tail()

,Year,Program,Six Digit,Program Category,Four Digit,Program Family,Two Digit,Institution Type,Institution,Campus,County,FIPS,Certificate,Diploma,Total Awards
513,2022-23,Sterile Processing Technology/Technician,51.1012,Clinical/Medical Laboratory Science/Research a...,51.10,Health Professions and Related Programs,51,TCAT,Murfreesboro,Main Campus,"Rutherford County, Tennessee",47149,13,0,13
514,2022-23,Child Care Provider/Assistant,19.0709,"Human Development, Family Studies, and Related...",19.07,Family and Consumer Sciences/Human Sciences,19,TCAT,Nashville,Main Campus,"Davidson County, Tennessee",47037,1,7,8
515,2022-23,Culinary Arts/Chef Training,12.0503,Culinary Arts and Related Services,12.05,"Culinary, Entertainment, and Personal Services",12,TCAT,Oneida,Morgan County Career and Technical Center,"Morgan County, Tennessee",47129,0,1,1
516,2022-23,Building/Property Maintenance,46.0401,"Building/Construction Finishing, Management, a...",46.04,Construction Trades,46,TCAT,Paris,Main Campus,"Henry County, Tennessee",47079,9,8,17
517,2022-23,Small Engine Mechanics and Repair Technology/T...,47.0606,Vehicle Maintenance and Repair Technologies/Te...,47.06,Mechanic and Repair Technologies/Technicians,47,TCAT,Paris,Main Campus,"Henry County, Tennessee",47079,1,0,1


In [145]:
comm = pd.read_csv('../../Data Downloads/TBR_CommunityColleges_20192020_20222023.csv', dtype = str)

In [146]:
comm.head()

,award_count,COLLEGE,Award_Type,Report_Year,CIP
0,2,Chattanooga,Certificate,2019 - 2020,110801
1,7,Chattanooga,Certificate,2019 - 2020,150000
2,15,Chattanooga,Certificate,2019 - 2020,150406
3,11,Chattanooga,Certificate,2019 - 2020,150612
4,4,Chattanooga,Certificate,2019 - 2020,150614


In [147]:
comm.reset_index(drop = True, inplace = True)

In [148]:
comm.columns

Index(['award_count', 'COLLEGE', 'Award_Type', 'Report_Year', 'CIP'], dtype='object')

In [149]:
comm = comm.drop_duplicates().fillna(0)

In [150]:
comm_pivot = pd.pivot_table(comm, index = ['COLLEGE', 'Report_Year', 'CIP'], columns = 'Award_Type', values = 'award_count', aggfunc = 'sum')

In [151]:
comm_pivot.head()

Award_Type                     Associates Certificate
COLLEGE     Report_Year CIP                          
Chattanooga 2019 - 2020 110103         29         NaN
                        110801        NaN           2
                        130101          9         NaN
                        140102         44         NaN
                        150000       4143           7

In [152]:
comm = comm_pivot.reset_index(drop = False)
comm = comm.rename_axis(None, axis=1)

In [153]:
comm = comm.fillna(0)
cols = ['Associates', 'Certificate']
comm[cols] = comm[cols].astype(float)

In [154]:
comm['Total Awards'] = comm['Associates'] + comm['Certificate']

In [155]:
comm['Two Digit'] = comm['CIP'].str.slice(start = 0, stop = 2)
comm['Four Digit'] = comm['Two Digit'] + '.' + comm['CIP'].str.slice(start = 2, stop = 4)
comm['Six Digit'] = comm['Four Digit'] + comm['CIP'].str.slice(start = 4)

In [156]:
comm = comm.merge(cip, on = 'Six Digit')

In [161]:
comm.head()

,Institution,Year,Associates,Certificate,Total Awards,Two Digit,Four Digit,Six Digit,Program Family,Program Category,Program,Institution Type,Campus,County,FIPS
0,Chattanooga,2019 - 2020,29.0,0.0,29.0,11,11.01,11.0103,Computer and Information Sciences and Support ...,"Computer and Information Sciences, General",Information Technology,Community College,Not Available for Community Colleges,"Hamilton County, Tennessee",47065
1,Chattanooga,2020 - 2021,18.0,0.0,18.0,11,11.01,11.0103,Computer and Information Sciences and Support ...,"Computer and Information Sciences, General",Information Technology,Community College,Not Available for Community Colleges,"Hamilton County, Tennessee",47065
2,Chattanooga,2021 - 2022,19.0,0.0,19.0,11,11.01,11.0103,Computer and Information Sciences and Support ...,"Computer and Information Sciences, General",Information Technology,Community College,Not Available for Community Colleges,"Hamilton County, Tennessee",47065
3,Chattanooga,2022 - 2023,27.0,0.0,27.0,11,11.01,11.0103,Computer and Information Sciences and Support ...,"Computer and Information Sciences, General",Information Technology,Community College,Not Available for Community Colleges,"Hamilton County, Tennessee",47065
4,Cleveland,2019 - 2020,15.0,0.0,15.0,11,11.01,11.0103,Computer and Information Sciences and Support ...,"Computer and Information Sciences, General",Information Technology,Community College,Not Available for Community Colleges,"Bradley County, Tennessee",47011


In [158]:
comm = comm.rename(columns = {'COLLEGE': 'Institution',
                                  'Report_Year': 'Year'})
comm = comm.drop(columns = 'CIP')
comm['Institution Type'] = 'Community College'
comm['Campus'] = 'Not Available for Community Colleges'

In [159]:
comm['County'] = comm['Institution'].map(communitycollegescounties)
comm['FIPS'] = comm['County'].map(geotogeoid)

In [160]:
boop = comm['FIPS'].str.split(pat = 'US', expand = True)
comm['FIPS'] = boop[1].str.strip()

In [163]:
comm = comm[['Year', 'Program', 'Six Digit', 'Program Category', 'Four Digit', 'Program Family', 'Two Digit', 'Institution Type', 'Institution', 
         'Campus', 'County', 'FIPS', 'Certificate', 'Associates', 'Total Awards'
        ]]

In [165]:
comm['Diploma'] = 'Not Applicable'
tcats['Associates'] = 'Not Applicable'

C:\Users\jmccall\AppData\Local\Temp\ipykernel_2656\908978047.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comm['Diploma'] = 'Not Applicable'


In [166]:
dfs = [comm, tcats]
data = pd.concat(dfs)

In [168]:
data = data[['Year', 'Program', 'Six Digit', 'Program Category', 'Four Digit', 'Program Family', 'Two Digit', 'Institution Type', 'Institution', 
         'Campus', 'County', 'FIPS', 'Certificate', 'Associates', 'Diploma', 'Total Awards'
        ]]

In [180]:
data.head()

,Year,Program,Six Digit,Program Category,Four Digit,Program Family,Two Digit,Institution Type,Institution,Campus,County,FIPS,Certificate,Associates,Diploma,Total Awards
0,2019-20,Information Technology,11.0103,"Computer and Information Sciences, General",11.01,Computer and Information Sciences and Support ...,11,Community College,Chattanooga,Not Available for Community Colleges,"Hamilton County, Tennessee",47065,0.0,29.0,Not Applicable,29.0
1,2020-21,Information Technology,11.0103,"Computer and Information Sciences, General",11.01,Computer and Information Sciences and Support ...,11,Community College,Chattanooga,Not Available for Community Colleges,"Hamilton County, Tennessee",47065,0.0,18.0,Not Applicable,18.0
2,2021-22,Information Technology,11.0103,"Computer and Information Sciences, General",11.01,Computer and Information Sciences and Support ...,11,Community College,Chattanooga,Not Available for Community Colleges,"Hamilton County, Tennessee",47065,0.0,19.0,Not Applicable,19.0
3,2022-23,Information Technology,11.0103,"Computer and Information Sciences, General",11.01,Computer and Information Sciences and Support ...,11,Community College,Chattanooga,Not Available for Community Colleges,"Hamilton County, Tennessee",47065,0.0,27.0,Not Applicable,27.0
4,2019-20,Information Technology,11.0103,"Computer and Information Sciences, General",11.01,Computer and Information Sciences and Support ...,11,Community College,Cleveland,Not Available for Community Colleges,"Bradley County, Tennessee",47011,0.0,15.0,Not Applicable,15.0


In [177]:
data['Year'] = data['Year'].replace({'2019 - 2020': '2019-20', 
                                    '2020 - 2021': '2020-21', 
                                    '2021 - 2022': '2021-22', 
                                    '2022 - 2023': '2022-23'})

In [178]:
data['Year'].unique()

array(['2019-20', '2020-21', '2021-22', '2022-23'], dtype=object)

In [179]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3300 entries, 0 to 517
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Year              3300 non-null   object
 1   Program           3300 non-null   object
 2   Six Digit         3300 non-null   object
 3   Program Category  3300 non-null   object
 4   Four Digit        3300 non-null   object
 5   Program Family    3300 non-null   object
 6   Two Digit         3300 non-null   object
 7   Institution Type  3300 non-null   object
 8   Institution       3300 non-null   object
 9   Campus            3300 non-null   object
 10  County            3287 non-null   object
 11  FIPS              3154 non-null   object
 12  Certificate       3300 non-null   object
 13  Associates        3300 non-null   object
 14  Diploma           3300 non-null   object
 15  Total Awards      3300 non-null   object
dtypes: object(16)
memory usage: 438.3+ KB


In [181]:
data.nunique()

Year                  4
Program             167
Six Digit           167
Program Category     82
Four Digit           82
Program Family       25
Two Digit            25
Institution Type      2
Institution          37
Campus              107
County               71
FIPS                 70
Certificate         200
Associates          154
Diploma             108
Total Awards        320
dtype: int64

In [182]:
data.to_csv('ALLTBR.csv', index = False)